In [ ]:
"""
1. get image
2. resize image
3. add resized image to base
4. send to replicate 
5. return result
"""

In [1]:
#resize image
import os
from PIL import Image

def resize_image(input_image_path):
    """
    Resize an image to 400x400 pixels and save with '_resized' suffix
    
    Args:
        input_image_path (str): Path to the input image
    
    Returns:
        str: Path to the resized image
    """
    # Open the image
    with Image.open(input_image_path) as img:
        # Resize the image to 400x400 pixels using LANCZOS resampling
        resized_img = img.resize((400, 400), Image.Resampling.LANCZOS)
        
        # Create new filename with '_resized' suffix
        name, ext = os.path.splitext(input_image_path)
        output_image_path = f"{name}_resized{ext}"
            
        # Save the resized image
        resized_img.save(output_image_path)
        
        return output_image_path

In [5]:
#add resized image to base
def add_image_to_base(resized_image_path, base_image_path="base.png"):
    """
    Place a resized image in the center of the base image
    
    Args:
        resized_image_path (str): Path to the resized image (400x400)
        base_image_path (str): Path to the base image (default: 'base.png')
    
    Returns:
        str: Path to the combined image
    """
    # Open both images
    with Image.open(base_image_path) as base_img, Image.open(resized_image_path) as top_img:
        # Convert images to RGBA if they aren't already
        base_img = base_img.convert('RGBA')
        top_img = top_img.convert('RGBA')
        
        # Calculate the center position
        x = (base_img.width - top_img.width) // 2
        y = (base_img.height - top_img.height) // 2
        
        # Create a copy of the base image
        combined = base_img.copy()
        
        # Paste the resized image onto the base image
        combined.paste(top_img, (x, y), top_img)
        
        # Create output filename - always use .png extension
        name = os.path.splitext(resized_image_path)[0]
        output_path = f"{name}_combined.png"
        
        # Convert to RGB if saving as JPEG
        if output_path.lower().endswith('.jpg') or output_path.lower().endswith('.jpeg'):
            combined = combined.convert('RGB')
        
        # Save the combined image
        combined.save(output_path)
        
        return output_path

In [6]:
# First resize your image
resized_path = resize_image('example.jpeg')

# Then add it to the base
combined_path = add_image_to_base(resized_path)

In [9]:
import replicate
from pathlib import Path

# Create the input dictionary with file paths
input = {
    "image": Path("example_resized_combined.png"),
    "mask": Path("mask.png"),
    "prompt": "Extend the image beyond"
}

# Run the model
output = replicate.run(
    "black-forest-labs/flux-fill-pro",
    input=input
)

# Save the output
with open("output.jpg", "wb") as file:
    file.write(output.read())